# NO NEED TO RUN THIS NOTEBOOK

# SEE FINAL DATA PROCESSING CSV's `All_Orders_TRAIN.csv` and `All_Orders_TEST.csv`

In [ ]:
import collections
import pathlib
import random
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss
from math import sqrt
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score

%matplotlib inline
# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

db_path = pathlib.Path('instacart_database.db')

# Create csv for each table

In [ ]:
table_dict = {1 : 'aisles', 2: 'departments', 3: 'order_details', 4: 'orders', 6: 'products'}

for table in table_dict:
    table_name = table
    statement = f'select * from {table_dict[table]}'
    conn = sqlite3.Connection(db_path)
    table_dict[table] = pd.read_sql(statement, conn)
    
    print(statement)

In [ ]:
aisles_df = table_dict[1]
departments_df = table_dict[2]
order_details_df = table_dict[3]
orders_df = table_dict[4]
products_df = table_dict[6]

In [ ]:
aisles_df.to_csv('..\\capstone_project\\aisles.csv', index = False)
departments_df.to_csv('..\\capstone_project\\departments.csv', index = False)
order_details_df.to_csv('..\\capstone_project\\order_details.csv', index = False)
orders_df.to_csv('..\\capstone_project\\orders.csv', index = False)
products_df.to_csv('..\\capstone_project\\products.csv', index = False)

# csv option instead of querying database everytime

In [ ]:
# aisles_df = pd.read_csv('aisles.csv')
# departments_df = pd.read_csv('departments.csv')
# order_details_df = pd.read_csv('order_details.csv')
# orders_df = pd.read_csv('orders.csv')
# products_df = pd.read_csv('products.csv')

In [ ]:
all_orders_sorted.head()

# Join necessary tables

Add product info to `order_details_df`

In [ ]:
order_details_plus_products = pd.merge(order_details_df, products_df, how = 'left', on = 'product_id')

Join `order_details_plus_products` with `orders`

In [ ]:
order_details_plus_products_plus_orders = pd.merge(order_details_plus_products, orders_df, how = 'left', on = 'order_id')

# Sort and reset index

Sort by `user_id` and `order_id` ascending

In [ ]:
all_orders_sorted = order_details_plus_products_plus_orders.sort_values(by = ['user_id', 'order_number'])

Drop `id` column

In [ ]:
all_orders_sorted_set_index = all_orders_sorted.drop(columns='id')

Reset the index to 0

In [ ]:
all_orders_sorted_reset_index = all_orders_sorted_set_index.reset_index()

Drop new `index` column

In [ ]:
all_orders_sorted = all_orders_sorted_reset_index.drop(columns='index')

# Reduce our data to x number of users 

In [ ]:
all_orders_sorted['user_id'].max()

In [ ]:
user_list = list(all_orders_sorted['user_id'].unique())

In [ ]:
len(user_list)

### We have 206209 Users -- Lets take as many as we can without surpassing 20,000 rows

In [ ]:
all_orders_sorted.iloc[20000]

### 20000 rows gets us to 152 users -- lets see if we can get to 250 users

In [ ]:
all_orders_sorted.iloc[38000]

### Looks like 38000 rows gets us to 250 users

In [ ]:
# df.loc[df['shield'] > 6]

all_orders_reduced = all_orders_sorted.loc[all_orders_sorted['user_id'] < 257]

In [ ]:
all_orders_reduced.describe()

### Create CSV with reduced data

In [ ]:
all_orders_reduced.to_csv('Data_Reduced.csv')

# Assign users to test and train values 
- We want to keep 10% of the users for a test set to mimic the final product (see how our model performs without seeing any previous user data) 

In [ ]:
# Find the bottome 90% of users to create train data
256 * .9 # = 230.4 


### Therefore we are going to section off (`user_id` <= 229) as train & (`user_id` >= 230) as test data

In [ ]:
# df.loc[df['shield'] > 6]

train_data = all_orders_reduced.loc[all_orders_reduced['user_id'] <= 229] # <- shape = (33697, 12)
test_data =  all_orders_reduced.loc[all_orders_reduced['user_id'] >= 230] # <- shape = (4484, 12)

# Export to individual csv's so we don't have to repeat this process

In [ ]:
train_data.to_csv('All_Orders_TRAIN.csv')
test_data.to_csv('All_Orders_TEST.csv')